In [1]:
# Import necessary libraries

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import balanced_accuracy_score, hamming_loss, f1_score, recall_score, roc_auc_score, accuracy_score
from sklearn.preprocessing import LabelEncoder

provided_weights_genetic_disorder = {
    'Mitochondrial genetic inheritance disorders': 0.48794813542417026,
    'Single-gene inheritance diseases': 0.6160580705934504,
    'Multifactorial genetic inheritance disorders': 0.8959937939823793
}

provided_weights_subclass_disorder = {
    'Leigh syndrome': 0.740510888236272,
    'Mitochondrial myopathy': 0.7799634288247355,
    'Cystic fibrosis': 0.8257328087770821,
    'Tay-Sachs': 0.8583698121571453,
    'Diabetes': 0.9084058292236937,
    'Hemochromatosis': 0.9319554496592232,
    "Leber's hereditary optic neuropathy": 0.9674738183631628,
    "Alzheimer's": 0.9926303540754696,
    'Cancer': 0.9949576106832161
}


In [2]:
# Load dataset
dataset= pd.read_csv('dataset.csv')
#print(dataset.columns)

# Replace 'Unknown' with NaN
dataset.replace('Unknown', np.nan, inplace=True)

# Eliminate samples with
dataset.dropna(inplace=True)


# Features to use
features = ['Patient Age', "Mother's age", "Father's age", 'No. of previous abortion',
    'White Blood cell count (thousand per microliter)']
targets = ['Genetic Disorder', 'Disorder Subclass']

# Select only the desired columns
selected_columns = features + targets
dataset = dataset[selected_columns]



# Encode categorical variables
label_encoder = LabelEncoder()

# Create a copy for encoding
dataset_encoded = dataset.copy()

quantitative_with_unknowns_or_ordered_columns = [
    'Patient Age', "Mother's age", "Father's age", 'No. of previous abortion',
    'White Blood cell count (thousand per microliter)']



# Encode target variables 'Genetic Disorder' and 'Disorder Subclass'
# Create mapping
genetic_disorder_mapping = {label: i for i, label in enumerate(dataset_encoded['Genetic Disorder'].unique()) if pd.notna(label)}
disorder_subclass_mapping = {label: i for i, label in enumerate(dataset_encoded['Disorder Subclass'].unique())}

# Replace each original value with its corresponding encoded value as per the mapping
dataset_encoded['Genetic Disorder'] = dataset_encoded['Genetic Disorder'].map(genetic_disorder_mapping)
dataset_encoded['Disorder Subclass'] = dataset_encoded['Disorder Subclass'].map(disorder_subclass_mapping)


# Correspond weights to classes
# Get values mapped
encoded_values_genetic_disorder = dataset_encoded['Genetic Disorder'].unique()
encoded_values_disorder_subclass = dataset_encoded['Disorder Subclass'].unique()

# Inverse mappings to get back the original names
inverse_genetic_disorder_mapping = {i: label for label, i in genetic_disorder_mapping.items()}
inverse_disorder_subclass_mapping = {i: label for label, i in disorder_subclass_mapping.items()}

# Map encoded values back to original names
names_genetic_disorder = [inverse_genetic_disorder_mapping[i] for i in encoded_values_genetic_disorder]
names_disorder_subclass = [inverse_disorder_subclass_mapping[i] for i in encoded_values_disorder_subclass]

# Associate weights with encoded values
weights_genetic_disorder = {encoded_value: provided_weights_genetic_disorder[name] for encoded_value, name in zip(encoded_values_genetic_disorder, names_genetic_disorder)}
weights_disorder_subclass = {encoded_value: provided_weights_subclass_disorder[name] for encoded_value, name in zip(encoded_values_disorder_subclass, names_disorder_subclass)}

# Combine both class weights into a dictionary
class_weights = {'Genetic Disorder': weights_genetic_disorder, 'Disorder Subclass': weights_disorder_subclass}


class_weights_genetic_disorder = {0: 0.48794813542417026, 1: 0.8959937939823793, 2: 0.6160580705934504}

class_weights_disorder_subclass = {0: 0.740510888236272, 1: 0.9084058292236937, 2: 0.7799634288247355, 
                                   3: 0.8257328087770821, 4: 0.8583698121571453, 5: 0.9674738183631628, 
                                   6: 0.9319554496592232, 7: 0.9926303540754696, 8: 0.9949576106832161}

# Allocate features and labels
X = dataset_encoded.drop(columns=['Genetic Disorder', 'Disorder Subclass'])
y = dataset_encoded[['Genetic Disorder', 'Disorder Subclass']]


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=1)

In [3]:

# DecisionTreeClassifier for genetic disorder
dt_genetic_disorder = DecisionTreeClassifier(
    criterion='gini', max_depth=10, max_features='log2', min_samples_leaf=1, min_samples_split=10, splitter='random',
    class_weight=class_weights_genetic_disorder
)

# DecisionTreeClassifier for disorder subclass
dt_disorder_subclass = DecisionTreeClassifier(
    criterion='entropy', max_depth=10, max_features=None, min_samples_leaf=4, min_samples_split=2, splitter='random',
    class_weight=class_weights_disorder_subclass
)

# Fit the models
dt_genetic_disorder.fit(X_train, y_train['Genetic Disorder'])  # Assuming genetic disorder labels are in the first column
dt_disorder_subclass.fit(X_train, y_train['Disorder Subclass'])  # Assuming disorder subclass labels start from the second column

# Predict using the trained models
y_pred_genetic_disorder = dt_genetic_disorder.predict(X_test)
y_pred_disorder_subclass = dt_disorder_subclass.predict(X_test)

# Combine the predictions into a single array if needed
y_pred = np.column_stack((y_pred_genetic_disorder, y_pred_disorder_subclass))


In [4]:
# y_test_new is DataFrames
y_test_columns = y_test.columns[:]
# Convert y_pred_new to a DataFrame
y_pred_df = pd.DataFrame(y_pred, columns=y_test_columns)

balanced_acc_scores = {}

for column in y_test_columns:
    y_test_column = y_test[column]
    y_pred_column = y_pred_df[column]
    
    balanced_acc_scores[column] = balanced_accuracy_score(y_test_column, y_pred_column)



In [5]:
# Generate classification report for each output separately

y_test_array = y_test.to_numpy()

hamming_loss_genetic_disorder = hamming_loss(y_test_array[:, 0], y_pred[:, 0])
hamming_loss_disorder_subclass = hamming_loss(y_test_array[:, 1], y_pred[:, 1])

f1_score_genetic_disorder = f1_score(y_test_array[:, 0], y_pred[:, 0], average='weighted')
f1_score_disorder_subclass = f1_score(y_test_array[:, 1], y_pred[:, 1], average='weighted')

recall_genetic_disorder = recall_score(y_test_array[:, 0], y_pred[:, 0], average='weighted')
recall_disorder_subclass = recall_score(y_test_array[:, 1], y_pred[:, 1], average='weighted')

accuracy_genetic_disorder = accuracy_score(y_test_array[:, 0], y_pred[:, 0])
accuracy_disorder_subclass = accuracy_score(y_test_array[:, 1], y_pred[:, 1])

balanced_accuracy_genetic_disorder = balanced_accuracy_score(y_test_array[:, 0], y_pred[:, 0])
balanced_accuracy_disorder_subclass = balanced_accuracy_score(y_test_array[:, 1], y_pred[:, 1])


# Use dataset without Cancer and Alzeimer

In [6]:
# Assuming your DataFrame is named dataset
filtered_dataset = dataset_encoded[(dataset['Disorder Subclass'] != 7) & (dataset['Disorder Subclass'] != 8)]

# Update your X and y with the filtered dataset
X_filtered = filtered_dataset.drop(columns=['Genetic Disorder', 'Disorder Subclass'])
y_filtered = filtered_dataset[['Genetic Disorder', 'Disorder Subclass']]


X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X_filtered, y_filtered, train_size=0.75,  random_state=1) # training as 75%

In [7]:
# DecisionTreeClassifier for genetic disorder
dt_genetic_disorder_new = DecisionTreeClassifier(
    criterion='gini', max_depth=10, max_features='log2', min_samples_leaf=1, min_samples_split=10, splitter='random',
    class_weight=class_weights_genetic_disorder
)

# DecisionTreeClassifier for disorder subclass
dt_disorder_subclass_new = DecisionTreeClassifier(
    criterion='entropy', max_depth=10, max_features=None, min_samples_leaf=4, min_samples_split=2, splitter='random',
    class_weight=class_weights_disorder_subclass
)

# Fit the models
dt_genetic_disorder.fit(X_train, y_train['Genetic Disorder'])  # Assuming genetic disorder labels are in the first column
dt_disorder_subclass.fit(X_train, y_train['Disorder Subclass'])  # Assuming disorder subclass labels start from the second column

# Predict using the trained models
y_pred_genetic_disorder_new = dt_genetic_disorder.predict(X_test)
y_pred_disorder_subclass_new = dt_disorder_subclass.predict(X_test)

# Combine the predictions into a single array if needed
y_pred_new = np.column_stack((y_pred_genetic_disorder_new, y_pred_disorder_subclass_new))

In [8]:
# y_test_new is DataFrames
y_test_columns_new = y_test_new.columns[:]
# Convert y_pred_new to a DataFrame
y_pred_df_new = pd.DataFrame(y_pred_new, columns=y_test_columns_new)

balanced_acc_scores_new = {}

for column in y_test_columns:
    y_test_column_new = y_test_new[column]
    y_pred_column_new = y_pred_df_new[column]
    
    balanced_acc_scores_new[column] = balanced_accuracy_score(y_test_column_new, y_pred_column_new)



In [9]:
# Generate classification report for each output separately
y_test_array_new = y_test_new.to_numpy()

hamming_loss_genetic_disorder_new = hamming_loss(y_test_array_new[:, 0], y_pred_new[:, 0])
hamming_loss_disorder_subclass_new = hamming_loss(y_test_array_new[:, 1], y_pred_new[:, 1])

f1_score_genetic_disorder_new = f1_score(y_test_array_new[:, 0], y_pred_new[:, 0], average='weighted')
f1_score_disorder_subclass_new = f1_score(y_test_array_new[:, 1], y_pred_new[:, 1], average='weighted')

recall_genetic_disorder_new = recall_score(y_test_array_new[:, 0], y_pred_new[:, 0], average='weighted')
recall_disorder_subclass_new = recall_score(y_test_array_new[:, 1], y_pred_new[:, 1], average='weighted')

balanced_accuracy_genetic_disorder_new = balanced_accuracy_score(y_test_array_new[:, 0], y_pred_new[:, 0])
balanced_accuracy_disorder_subclass_new = balanced_accuracy_score(y_test_array_new[:, 1], y_pred_new[:, 1])

accuracy_genetic_disorder_new = accuracy_score(y_test_array_new[:, 0], y_pred_new[:, 0])
accuracy_disorder_subclass_new = accuracy_score(y_test_array_new[:, 1], y_pred_new[:, 1])


In [10]:
print("Classification for Genetic Disorder:")

print("Hamming Loss for Genetic Disorder:", hamming_loss_genetic_disorder)
print("F1 Score for Genetic Disorder:", f1_score_genetic_disorder)
print("Recall for Genetic Disorder:", recall_genetic_disorder)
print("Accuracy for Genetic Disorder:", accuracy_genetic_disorder)
print("Balanced Accuracy for Genetic Disorder:", balanced_accuracy_genetic_disorder)

print("\nClassification for Disorder Subclass:")

print("Hamming Loss for Disorder Subclass:", hamming_loss_disorder_subclass)
print("F1 Score for Disorder Subclass:", f1_score_disorder_subclass)
print("Recall for Disorder Subclass:", recall_disorder_subclass)
print("Balanced Accuracy for Disorder Subclass:", balanced_accuracy_disorder_subclass)
print("Accuracy for Disorder Subclass:", accuracy_disorder_subclass)


print("\n ------------ Without Cancer and Alzeimer samples --------------")

print("Classification for Genetic Disorder:")

print("\nHamming Loss for Genetic Disorder:", hamming_loss_genetic_disorder_new)
print("F1 Score for Genetic Disorder:", f1_score_genetic_disorder_new)
print("Recall for Genetic Disorder:", recall_genetic_disorder_new)
print("Balanced Accuracy for Genetic Disorder:", balanced_accuracy_genetic_disorder_new)
print("Accuracy for Genetic Disorder:", accuracy_genetic_disorder_new)

print("\nClassification for Disorder Subclass:")

print("Hamming Loss for Disorder Subclass:", hamming_loss_disorder_subclass_new)
print("F1 Score for Disorder Subclass:", f1_score_disorder_subclass_new)
print("Recall for Disorder Subclass:", recall_disorder_subclass_new)
print("Balanced Accuracy for Disorder Subclass:", balanced_accuracy_disorder_subclass_new)
print("Accuracy for Disorder Subclass:", accuracy_disorder_subclass_new)


Classification for Genetic Disorder:
Hamming Loss for Genetic Disorder: 0.5418326693227091
F1 Score for Genetic Disorder: 0.4632332113552061
Recall for Genetic Disorder: 0.4581673306772908
Accuracy for Genetic Disorder: 0.4581673306772908
Balanced Accuracy for Genetic Disorder: 0.3891343467675498

Classification for Disorder Subclass:
Hamming Loss for Disorder Subclass: 0.8406374501992032
F1 Score for Disorder Subclass: 0.16130291452767645
Recall for Disorder Subclass: 0.1593625498007968
Balanced Accuracy for Disorder Subclass: 0.08856456724103784
Accuracy for Disorder Subclass: 0.1593625498007968

 ------------ Without Cancer and Alzeimer samples --------------
Classification for Genetic Disorder:

Hamming Loss for Genetic Disorder: 0.5776892430278885
F1 Score for Genetic Disorder: 0.4155330557469371
Recall for Genetic Disorder: 0.42231075697211157
Balanced Accuracy for Genetic Disorder: 0.3028474648036414
Accuracy for Genetic Disorder: 0.42231075697211157

Classification for Disorder